## Jetson Nano - Object Detection Headless (from a Jupyter Notebook) ##

* Install JetCard https://github.com/NVIDIA-AI-IOT/jetcard
* Connect to your network and navigate to http://your-jetson-ip:8888/
* Install all necessary pre-reqs for jetson inference demos in the Terminal https://github.com/dusty-nv/jetson-inference/blob/master/docs/building-repo-2.md

In [ ]:
import ipywidgets
from IPython.display import display
import jetson.inference
import jetson.utils
import cv2

# turn numpy array into a jpeg
def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])


# load detect net
net = jetson.inference.detectNet("ssd-mobilenet-v2", threshold=0.5)

camera = jetson.utils.gstCamera(1280, 720, "/dev/video0")
image_widget = ipywidgets.Image(format='jpeg')
text_widget = ipywidgets.Text(value='Hello World')
display(image_widget,text_widget)

while 1:
    # get new image, zero copy needs to be 1 so we can access memory from cpu
    img, width, height = camera.CaptureRGBA(zeroCopy=1)
    
    # detect, also appends image with found objects
    detections = net.Detect(img, width, height)
    
    # we need a jpeg to display, depth is 4 because of alpha channel
    image = jetson.utils.cudaToNumpy(img,width, height, 4)    
    image_widget.value = bgr8_to_jpeg(image)
    # lets put the found things in a text field just because
    text_widget.value = " ".join([net.GetClassDesc(d.ClassID) for d in detections])



Image(value=b'', format='jpeg')

Text(value='Hello World')